<a href="https://colab.research.google.com/github/anushk218/codsoft/blob/main/Task_5_creditcard(codsoft).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [ ]:
df = pd.read_csv('/content/creditcard.csv.zip')
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [ ]:
df.shape

(284807, 31)

In [ ]:
df.dtypes

,0
Time,float64
V1,float64
V2,float64
V3,float64
V4,float64
V5,float64
V6,float64
V7,float64
V8,float64
V9,float64


In [ ]:
df.isnull().sum()

,0
Time,0
V1,0
V2,0
V3,0
V4,0
V5,0
V6,0
V7,0
V8,0
V9,0


In [ ]:
fraud_count = df['Class'].value_counts()
fraud_rate=100*fraud_count[1]/df.shape[0]
fraud_rate_series = pd.Series([fraud_rate], index=[1])
fraud_data=pd.concat([fraud_count,fraud_rate_series],axis=1).reset_index()
fraud_data.columns=['Class','Count','Percentage']
fraud_data

,Class,Count,Percentage
0,0,284315,NaN
1,1,492,0.172749


In [ ]:
df_fraud = df[df['Class']==1]
df_non_fraud = df[df['Class']==0]
df_non_fraud_sample = df_non_fraud.sample(df_fraud.shape[0],replace=False, random_state=101)
df_balanced = pd.concat([df_fraud,df_non_fraud_sample],axis=0).sample(frac=1,replace=False, random_state=101).reset_index(drop=True)
df_balanced

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,78705.0,-0.192520,1.134870,-0.882093,-0.421884,3.046217,3.123977,0.354874,0.836889,-0.773495,...,-0.049284,-0.265028,-0.316805,0.850044,0.408752,-0.352036,0.063642,0.056461,1.00,0
1,79541.0,-1.443178,0.238941,2.736067,0.974400,-1.827731,1.003379,-1.087927,0.840279,-0.549321,...,-0.353073,-0.153613,-0.238456,0.570593,0.242218,-0.134805,0.082475,0.068238,50.00,0
2,35953.0,-4.844372,5.649439,-6.730396,5.252842,-4.409566,-1.740767,-6.311699,3.449167,-5.416284,...,1.194888,-0.845753,0.190674,-0.216443,-0.325033,-0.270328,0.210214,0.391855,111.70,1
3,40918.0,-3.140260,3.367342,-2.778931,3.859701,-1.159518,-0.721552,-4.195342,-0.598346,-2.870145,...,2.452339,-0.292963,-0.189330,-0.166482,0.038040,-0.015477,0.776691,0.397557,0.76,1
4,159844.0,-0.408111,3.132944,-3.098030,5.803893,0.890609,-0.501474,-0.440054,0.591828,-3.267693,...,0.098482,-0.538375,-0.217989,-1.042657,0.314389,0.543244,0.233851,0.119603,45.51,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,144884.0,1.904250,0.096942,-1.671555,1.418529,0.495685,-1.058180,0.809534,-0.497326,0.026363,...,0.099239,0.418947,-0.083604,-0.043614,0.481224,-0.496105,-0.020044,-0.048556,76.00,0
980,51975.0,-0.625962,1.123573,1.727483,-0.093700,0.086500,-0.566385,0.906038,-0.203707,-0.189870,...,-0.254691,-0.359737,0.014452,0.397009,-0.261404,0.054901,0.236376,-0.043926,13.99,0
981,102114.0,-28.255053,21.467203,-26.871339,11.737436,-17.999630,6.065901,-41.506796,-38.987263,-13.434066,...,-21.453736,8.361985,4.909111,0.098328,-1.508739,0.008711,-5.407824,-0.183811,0.00,1
982,44617.0,-1.218631,1.246967,0.392905,-2.257589,0.573492,-0.033315,-0.224566,-2.790923,-0.255516,...,-1.204149,0.386046,-0.156786,-0.938677,-0.112085,1.132287,-0.344618,0.123007,9.99,0


In [ ]:
fraud_count = df['Class'].value_counts()
fraud_rate=100*fraud_count[1]/df.shape[0]
fraud_rate_series = pd.Series([fraud_rate], index=[1])
fraud_data=pd.concat([fraud_count,fraud_rate_series],axis=1).reset_index()
fraud_data.columns=['Class','Count','Percentage']
fraud_data

,Class,Count,Percentage
0,0,284315,NaN
1,1,492,0.172749


In [ ]:
x_train,x_test,y_train,y_test = train_test_split(df_balanced.drop('Class',axis=1),df_balanced['Class'],test_size=0.3,random_state=101)
print(f'x_train.shape,x_test.shape,y_train.shape,y_test.shape')

x_train.shape,x_test.shape,y_train.shape,y_test.shape


In [ ]:
randomForestModel=Pipeline([
    ('scaler',StandardScaler()),
    ('classifier',RandomForestClassifier())
])
randomForestModel.fit(x_train,y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier', RandomForestClassifier())])

In [ ]:
y_pred_logis=randomForestModel.predict(x_test)
y_pred_logis

array([1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 0])

In [ ]:
cr=classification_report(y_test,y_pred_logis)
print(cr)

              precision    recall  f1-score   support

           0       0.89      0.99      0.94       146
           1       0.99      0.88      0.93       150

    accuracy                           0.94       296
   macro avg       0.94      0.94      0.94       296
weighted avg       0.94      0.94      0.94       296



In [ ]:
with open('./model.pkl','wb') as f:
  pickle.dump(randomForestModel,f)